In [1]:
%cd C:/Users/wangj337/Google Drive/Courses/IntroAI-CS50/6Language/parser

C:\Users\wangj337\Google Drive\Courses\IntroAI-CS50\6Language\parser


In [1]:
%cd C:/Users/wangj337/Google Drive/Courses/IntroAI-CS50/6Language/src6
%run sentiment/sentiment.py "sentiment/corpus"

C:\Users\wangj337\Google Drive\Courses\IntroAI-CS50\6Language\src6


s:  the product is overpriced


Positive: 0.6658
Negative: 0.3342


In [2]:
%run parser.py "sentences/7.txt"

She never said a word until we were at the door here.

                                  S                                     
                __________________|_____________                         
               |                  |             S                       
               |                  |     ________|_______                 
               |                  |    |                VP              
               |                  |    |             ___|____________    
               S                  |    |            VP               |  
      _________|____              |    |    ________|___             |   
     NP             VP            |    |   |            PP           |  
  ___|____      ____|___          |    |   |     _______|___         |   
 NP      AVP   |        NP        |    NP  VP   |           NP      AVP 
 |        |    |     ___|___      |    |   |    |        ___|___     |   
 N       Adv   V   Det      N    Conj  N   V    P      Det     

In [21]:
# %load parser.py
import nltk
import sys

TERMINALS = """
Adj -> "country" | "dreadful" | "enigmatical" | "little" | "moist" | "red"
Adv -> "down" | "here" | "never"
Conj -> "and" | "until"
Det -> "a" | "an" | "his" | "my" | "the"
N -> "armchair" | "companion" | "day" | "door" | "hand" | "he" | "himself"
N -> "holmes" | "home" | "i" | "mess" | "paint" | "palm" | "pipe" | "she"
N -> "smile" | "thursday" | "walk" | "we" | "word"
P -> "at" | "before" | "in" | "of" | "on" | "to"
V -> "arrived" | "came" | "chuckled" | "had" | "lit" | "said" | "sat"
V -> "smiled" | "tell" | "were"
"""

NONTERMINALS = """
S -> NP VP | S Conj S | S Conj VP
NP -> N | Det N | Det AP N | NP PP | AP NP | NP AVP
VP -> V | V NP | VP PP | VP AVP | AVP VP
PP -> P NP
AP -> Adj | Adj AP 
AVP -> Adv
""" 



#


grammar = nltk.CFG.fromstring(NONTERMINALS + TERMINALS)
parser = nltk.ChartParser(grammar)



In [6]:

def main():

    # If filename specified, read sentence from file
    if len(sys.argv) == 2:
        with open(sys.argv[1]) as f:
            s = f.read()

    # Otherwise, get sentence as input
    else:
        s = input("Sentence: ")

    # Convert input into list of words
    s = preprocess(s)

    # Attempt to parse sentence
    try:
        trees = list(parser.parse(s))
    except ValueError as e:
        print(e)
        return
    if not trees:
        print("Could not parse sentence.")
        return

    # Print each tree with noun phrase chunks
    for tree in trees:
        tree.pretty_print()

        print("Noun Phrase Chunks")
        for np in np_chunk(tree):
            print(" ".join(np.flatten()))



In [7]:
def preprocess(sentence):
    """
    Convert `sentence` to a list of its words.
    Pre-process sentence by converting all characters to lowercase
    and removing any word that does not contain at least one alphabetic
    character.
    """
    
    contents = []
    contents.extend([
        word.lower() for word in
        nltk.word_tokenize(sentence)
        if any(c.isalpha() for c in word)
    ])
    return contents
    


def np_chunk(tree):
    """
    Return a list of all noun phrase chunks in the sentence tree.
    A noun phrase chunk is defined as any subtree of the sentence
    whose label is "NP" that does not itself contain any other
    noun phrases as subtrees.
    """
    np = []
    for a in tree.subtrees():
        if a.label() == "NP":
            total = [sub.label() for sub in a.subtrees()]
            if total.count("NP") == 1:
                np.append(a)         
    return np

In [5]:
s = "I had a country walk on Thursday and came home in a dreadful mess."

# Convert input into list of words
s = preprocess(s)
trees = list(parser.parse(s))
for np in np_chunk(trees[0]):
    print(" ".join(np.flatten()))

trees[0].pretty_print()
    

i
walk
thursday
home
mess
                                            S                                                     
      ______________________________________|_________________________                             
     S                                             |                  |                           
  ___|_____________                                |                  |                            
 |                 VP                              |                  VP                          
 |            _____|__________________             |          ________|_______                     
 |           VP                       |            |         |                PP                  
 |    _______|_____                   |            |         |         _______|_____               
 |   |             NP                 |            |         |        |             NP            
 |   |    _________|_____             |            |         |        |    ____

In [22]:
s = "I had a little moist red paint in the palm of my hand.."

# Convert input into list of words
s = preprocess(s)
trees = list(parser.parse(s))
print(s)
#print(trees)
tree = trees[0]

['i', 'had', 'a', 'little', 'moist', 'red', 'paint', 'in', 'the', 'palm', 'of', 'my', 'hand']


In [9]:
tree.label()
labels = [sub.label() for sub in tree.subtrees()]
print(labels.count("NP"))
print(labels.count("S"))
print(labels.count("VP"))

3
1
2


In [23]:
for np in np_chunk(tree):
    print(" ".join(np.flatten()))

i
a little moist red paint
the palm
my hand


In [67]:
np_chunk(tree)

[Tree('NP', [Tree('NPX', [Tree('N', ['i'])])]),
 Tree('NP', [Tree('Det', ['the']), Tree('NPX', [Tree('N', ['palm'])])]),
 Tree('NP', [Tree('Det', ['my']), Tree('NPX', [Tree('N', ['hand'])])])]

In [24]:
tree.pretty_print()

              S                                                         
  ____________|____________________                                      
 |                                 VP                                   
 |    _____________________________|____                                 
 |   |                                  NP                              
 |   |                         _________|____________________            
 |   |                        NP                             |          
 |   |                ________|_____________                 |           
 |   |               NP                     |                |          
 |   |    ___________|_____________         |                |           
 |   |   |           AP            |        |                |          
 |   |   |     ______|____         |        |                |           
 |   |   |    |           AP       |        PP               PP         
 |   |   |    |       ____|___     |     ___|

In [12]:
for a in trees[0]:
    a.pretty_print()


                 S                            
  _______________|________                     
 |                        VP                  
 |            ____________|________            
 |           VP                    |          
 |    _______|___                  |           
 |   |           NP                PP         
 |   |    _______|___           ___|___        
 |   |   |          NPX        |       NP     
 |   |   |        ___|____     |    ___|___    
 NP  |   |       AP      NPX   |   |      NPX 
 |   |   |       |        |    |   |       |   
 N   V  Det     Adj       N    P  Det      N  
 |   |   |       |        |    |   |       |   
 i  had  a      red     paint  in  my     hand

                 S                            
  _______________|________                     
 |                        VP                  
 |            ____________|________            
 |           VP                    |          
 |    _______|___                  |           
 |

In [48]:
for tree in trees:
    tree.pretty_print()

     S                                                                      
  ___|___                                                                    
 |       VP                                                                 
 |    ___|___________                                                        
 |   |               NP                                                     
 |   |    ___________|__________________                                     
 |   |   |                             NPX                                  
 |   |   |            __________________|_______________                     
 |   |   |           |                                 NPX                  
 |   |   |           |                       ___________|________            
 |   |   |           |                     NPX                   |          
 |   |   |           |              ________|___                 |           
 |   |   |           AP            |            PP               PP   

In [ ]:
s = input("Sentence: ")
s

In [29]:
contents = []
contents.extend([
    word.lower() for word in
    nltk.word_tokenize(s)
    if any(c.isalpha() for c in word)
])
contents

['i', 'have', 'a', 'cat']

In [16]:
nltk.word_tokenize(s)

['I', 'have', 'a', 'cat']

In [28]:
sentence = input("Sentence: ").split()
sentence

Sentence:  I have a cat!


['I', 'have', 'a', 'cat!']

In [21]:
c = "92yrs"
print(c.isalpha())
c = "have"
c.isalpha()

False


True

In [23]:
nltk.ngrams(contents, 1)

In [26]:
ngrams = Counter(nltk.ngrams(contents, 1))
ngrams

Counter({('i',): 1, ('have',): 1, ('a',): 1, ('cat',): 1})